In [308]:
# load modules
import netCDF4 as nc
import glob
import numpy as np
import datetime
import os

In [309]:
# system timezone to UTC
# everything will be done in UTC
# UTC = CET-1h/CEST-2h
os.environ['TZ'] = 'UTC'

In [310]:
# date of interest
# need to find correct file path
doi = '2021-06-20' # year-month-day

In [311]:
# list the to file(s) at the doi
y_oi = doi.split('-')[0]
m_oi = doi.split('-')[1]
d_oi = doi.split('-')[2]

fnames = glob.glob('/Volumes/Elements/data/msevi_rss/raw_netcdf/'+y_oi+'/'+m_oi+'/'+d_oi+'/*')
fnames.sort()

In [313]:
# for loop -> loop over different files
# looping over all files is available in the .py script (msevi_nat_to_netcdf.py)
# in this notebook: i = [from 0 to number of files]
i = 9

In [314]:
# get the i-th filename 
fname = fnames[i]
ftext = fnames[i]

In [316]:
# get date and time information of the file
ftext = ftext.split("/")[9]
ftext = ftext.split(".")[1]
ftext = ftext.split("-")[1]

year = list(ftext)[0]+list(ftext)[1]+list(ftext)[2]+list(ftext)[3]
month = list(ftext)[4]+list(ftext)[5]
day = list(ftext)[6]+list(ftext)[7]
hour = list(ftext)[8]+list(ftext)[9]
minute = list(ftext)[10]+list(ftext)[11]
second = list(ftext)[12]+list(ftext)[13]

In [288]:
# open netCDF file
ds = nc.Dataset(fname)

In [289]:
# get IR_108 and WV_062 channel data in counts unit
ir_counts = ds.variables['IR_108'][:]
wv_counts = ds.variables['WV_062'][:]


### Coefficients for conversion </p>
**WV_062**
- cal_slope: 0.0083
- cal_offset: -0.424
- fi: 88.624939 

**IR_108**
- cal_slope: 0.2156
- cal_offset: -10.4
- fi: 65.836404


In [290]:
# convert counts to spectral radiance (mW m-2 sr-1 cm-1)
ir_specrad = ir_counts*0.2156+(-10.4)
wv_specrad = wv_counts*0.0083+(-0.424)

In [291]:
# convert spectral radiance to radiance
ir_rad = ir_specrad*65.836/1000
wv_rad = wv_specrad*88.625/1000

In [292]:
# calculating outgoing longwave radiation (OLR) (Singh et al., 2007)
olr = 12.94*ir_rad+16.5*wv_rad+((10.09*wv_rad)/ir_rad)+((12.94*wv_rad)/(ir_rad+0.39))+77.47

In [293]:
# put array into a big OLR dataset

# if i = 0 in the for loop:
# https://stackoverflow.com/questions/53882600/numpy-append-multiple-different-np-arrays-in-one-axis-of-existing-np-array
OLR = np.ma.asanyarray([olr]*len(fnames))

# if i >= 1 in the for loop:
OLR[i] = olr*20 

In [294]:
############
# SENSING TIME AS UNIX-TIMESTAMP
# seconds since 1970-01-01 00:00:00 (UTC)
dt_str = year+'-'+month+'-'+day+' '+hour+':'+minute+':'+second
dt_object = datetime.datetime.fromisoformat(dt_str)
timestamp = datetime.datetime.timestamp(dt_object)

In [295]:
# put array into a big time array

# if i = 0 in the for loop:
times = np.ma.array([int(timestamp)]*len(fnames))

# if i >=1 in the for loop:
times[i] = timestamp

['2021-06-20',
 '2021-06-21',
 '2021-06-22',
 '2021-07-04',
 '2021-07-13',
 '2021-07-15',
 '2021-07-16',
 '2021-07-17',
 '2021-07-24',
 '2021-07-25',
 '2021-07-26',
 '2021-07-28',
 '2021-07-30']

In [296]:
## !! END OF FOR LOOP HERE !!

In [297]:
############
# GET X,Y DIMENSIONS
# define latitude and longitude array
lat = ds.variables['latitude'][:]
lon = ds.variables['longitude'][:]
lat_dim = np.ma.masked_array(np.arange(lat.min(),lat.max()+0.001,(lat.max()-lat.min())/(lat.shape[1]-1)))
lon_dim = np.ma.masked_array(np.arange(lon.min(),lon.max()+0.001,(lon.max()-lon.min())/(lon.shape[0]-1)))

In [298]:
############
# CREATE THE NETCDF FILE
nc_fn = '/Volumes/Elements/data/msevi_rss/olr/'+y_oi+'/'+m_oi+'/'+'OLR_'+year+month+day+'.nc'
nc_ds = nc.Dataset(nc_fn, 'w', format = 'NETCDF4')

In [299]:
# define dimensions
time = nc_ds.createDimension('time', len(times))
lat = nc_ds.createDimension('lat', len(lat_dim))
lon = nc_ds.createDimension('lon', len(lon_dim))

In [300]:
# define variables
olr = nc_ds.createVariable('olr', 'f4', ('time', 'lat', 'lon',))
time = nc_ds.createVariable('time', 'f4', ('time',))
lat = nc_ds.createVariable('lat', 'f4', ('lat',))
lon = nc_ds.createVariable('lon', 'f4', ('lon',))

In [301]:
# define units
olr.units = 'W m^-2'
time.units = 'seconds since 1970-1-1'
lat.units = 'degrees_north'
lon.units = 'degrees_east'

In [302]:
# define other names
olr.long_name = 'OLR'
time.standard_name = 'time'
lat.standard_name = 'latitude'
lon.standard_name = 'longitude'

In [303]:
# define axis
time.axis = 'T'
lat.axis = 'Y'
lon.axis = 'X'

In [304]:
# define others
time.calendar = 'gregorian'
lat.datum = 'WGS84'
lon.datum = 'WGS84'

In [305]:
# define spacing
lat.spacing = str((lat_dim[len(lat_dim)-1]-lat_dim[0])/(len(lat_dim)-1))
lon.spacing = str((lon_dim[len(lon_dim)-1]-lon_dim[0])/(len(lon_dim)-1))

In [306]:
# assign data to variables
lat[:] = lat_dim
lon[:] = lon_dim
time[:] = times
olr[:] = OLR

In [307]:
# close the netCDF
nc_ds.close()